Todo list:
- ~~book reading function~~
- morphologic analysis:
    - ~~lexical diversity calculation function~~
    - ~~parts of speech counting function (by word's initial form)~~
    - ~~parts of speech counting function (by tags, part of speech types)~~
    - ~~text dynamics calculation functiion~~
    - *top-1 for every speech part
- syntax analysis
    - reduce the amount of text
    - ~~most popular root word~~
- semantic analysis
    - stop-word cleanup
    - *automatic tag generation
- graphic demonstration
    - tag cloud

# Imports and installs

In [1]:
%pip install -q nltk pymorphy2
import nltk
import pymorphy2
import spacy
import numpy as np

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nlp = spacy.load('ru_core_news_sm')

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to /home/andrew/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/andrew/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/andrew/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Defining a few constants

In [2]:
PUNCT = ('.',',',':',';','\'','"','-','(',')','!','?','...','$','№')

# Preprocess class definition

In [3]:
class Preproc:
    def __init__(self, book_name):
        self.text = self.get_book_text(book_name)
        self.tokens = nltk.word_tokenize(self.text)
        self.morph = pymorphy2.MorphAnalyzer()
        self.nlp = spacy.load('ru_core_news_sm')
        self.make_tags()
    
    def process_stopwords(self, overwrite=False):
        stopwords = stopwords.words("russian")
        if overwrite:
            pass
        else:
            self.tokens_no_stopwords = [token for token in self.tokens if token not in stopwords]

    def get_book_text(name):
        with open(name, "r") as raw:
            text = ""
            for t in raw.readlines():
                text += t+"\n"
            return text
    
    def lex_diversity(self):
        unique_words_set = set(self.tokens)
        return len(unique_words_set) / len(self.tokens)

    def make_tags(self):
        self.tagged = []
        for token in self.tokens:
            self.tagged.append((token, self.morph.parse(token)[0].tag.POS))
            # spacy find all собственные имена
        return self.tagged

    def text_dynamics(total_tags: list):
        verbs = total_tags.count("VERB")
        return verbs / len(total_tags)

    def text_dynamics_dict(self):
        self.count_speech_parts()
        verbs = self.tags_count["VERB"]
        total = 0
        for i in self.tags_count.keys():
            total += self.tags_count[i]
        return verbs/total
    
    def count_speech_parts(self):
        self.make_tags()
        res = dict()
        for token in self.tagged:
            if (token[1] in res.keys()) :
                res[token[1]] += 1
            else:
                res[token[1]] = 1
        self.tags_count = res

    def count_unique_words(self):
        self.make_tags()
        res = dict()
        for token in self.tagged:
            if not (token[1] in ("CONJ", "PREP", "PRCL")):
                if token[0] in res.keys():
                    res[token[0]] += 1
                else:
                    res[token[0]] = 1
        return res
    
    @staticmethod
    def compare_books(self, book1, book2):
        text1 = self.get_book_text(book1)
        text2 = self.get_book_text(book2)

        doc1 = self.nlp(text1)
        doc2 = self.nlp(text2)
        return (doc1.similarity(doc2), np.dot(doc1.vector, doc2.vector) / (np.linalg.norm(doc1.vector) * (np.linalg.norm(doc2.vector))))

# Text file reading function

In [4]:
def get_book(name):
    with open(name, "r") as raw:
        text = ""
        for t in raw.readlines():
            text += t+"\n"
        return text

# Lexical diversity calculation function

In [5]:
def lex_diversity(word_list):
    unique_words_set = set(word_list)
    return len(unique_words_set) / len(word_list)

# Text dynamics calculation function

In [6]:
def text_dynamics(total_tags: list):
    verbs = total_tags.count("VERB")
    return verbs / len(total_tags)

def text_dynamics_dict(tags_dict):
    verbs = tags_dict["VERB"]
    total = 0
    for i in tags_dict.keys():
        total += tags_dict[i]
    return verbs/total

# Morph analyzer init

In [7]:
morph = pymorphy2.MorphAnalyzer()
def tagger(tokenized):
    res = []
    for token in tokenized:
        res.append((token, morph.parse(token)[0].tag.POS))
        # spacy find all собственные имена
    return res

# Parts of speech counting function

In [8]:
# counter to count speech parts
def count_speech_parts(tagged):
    res = dict()
    for token in tagged:
        if (token[1] in res.keys()) :
            res[token[1]] += 1
        else:
            res[token[1]] = 1
    return res

def count_unique_words(tagged):
    res = dict()
    for token in tagged:
        if not (token[1] in ("CONJ", "PREP", "PRCL")):
            if token[0] in res.keys():
                res[token[0]] += 1
            else:
                res[token[0]] = 1
    return res

# Syntax analyzing functions

In [9]:
def count_root_speech_parts(doc):
    res = dict()
    for token in doc:
        if token.pos_ not in ("SPACE", "PUNCT"):
            if token.pos_ not in res.keys():
                res[token.pos_] = 1
            else:
                res[token.pos_] += 1
    return res

### Now onto the fun part
# Putting everything together

In [10]:
list_of_books = ["Похождения Чичикова (1921)","Белая гвардия (1922)","Роковые яйца (1924)","Собачье сердце (1925)","Мастер и маргарита (1929)","Театральный роман (1936)"]
for name in list_of_books:
    text = get_book(name)
    doc = nlp(text)

    tokenized_no_punct = nltk.tokenize.RegexpTokenizer(r"\w+").tokenize(text)
    tagged = tagger(tokenized_no_punct)
    with open(name+"_out", "w") as file:
        file.write(f"Lexical diversity: {str(lex_diversity(tokenized_no_punct))}\n")
        file.write("Top-3 words: \n")
        words_count = count_unique_words(tagger(tokenized_no_punct))
        words_count_sorted = dict(sorted(words_count.items(), key=lambda item: item[1]))

        file.write(f"\t{list(words_count_sorted.keys())[-1]}\n")
        file.write(f"\t{list(words_count_sorted.keys())[-2]}\n")
        file.write(f"\t{list(words_count_sorted.keys())[-3]}\n")

        speech_part_count = count_speech_parts(tagger(tokenized_no_punct))
        file.write(f"Text dynamics: {text_dynamics_dict(speech_part_count)}\n")

        root_count = count_root_speech_parts(doc)
        root_count_sorted = dict(sorted(root_count.items(), key=lambda item: item[1]))
        file.write(f"Top root speech part: {list(root_count_sorted.keys())[-1]}\n")

        # file.write("")

KeyboardInterrupt: 